### Imports

In [8]:
import os
import numpy as np

import tensorflow
from tensorflow import keras
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, concatenate, Activation, BatchNormalization, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
import tensorflow.keras.backend as K

In [9]:
print(tensorflow.__version__)
print(tensorflow.keras.__version__)

2.1.0
2.2.4-tf


### Settings

In [10]:
keras.backend.set_image_data_format('channels_last')  

img_rows = 240
img_cols = 240
smooth = 1.

### Loss Function Definition

In [11]:
def dice_coeff(y_true, y_pred):
    y_true_f = keras.backend.flatten(y_true)
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coeff_loss(y_true, y_pred):
    return -dice_coeff(y_true, y_pred)

### UNet Architecture Definition

In [12]:
def DoubleConv2D(n_filters, input, activation='relu', padding='same'):
    conv_output_a = Conv2D(n_filters, (3, 3), padding = padding)(input)
    bnorm_a = BatchNormalization()(conv_output_a)
    activated_a = Activation(activation)(bnorm_a)
    conv_output_b = Conv2D(n_filters, (3, 3), padding = padding)(activated_a)
    bnorm_b = BatchNormalization()(conv_output_b)
    activated_b = Activation(activation)(bnorm_b)
    return activated_b

def ContractingBlock2D(n_filters, input):
    double_conv_output = DoubleConv2D(n_filters, input)
    pool = MaxPooling2D(pool_size=(2, 2))(double_conv_output)
    return double_conv_output, pool

def ExpandingBlock2D(n_filters, upconv_input, concat_input):
    upsamp = Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding = 'same')(upconv_input)
    conv_input = concatenate([upsamp, concat_input], axis=3)
    double_conv_output = DoubleConv2D(n_filters, conv_input)
    return double_conv_output 

def UNet():
    input_tensor = Input((img_rows, img_cols, 1))

    conv_1, pool_1 = ContractingBlock2D(32, input_tensor)
    conv_2, pool_2 = ContractingBlock2D(64, pool_1)
    conv_3, pool_3 = ContractingBlock2D(128, pool_2)
    conv_4, pool_4 = ContractingBlock2D(256, pool_3)

    conv_5 = DoubleConv2D(512, pool_4)

    conv_6 = ExpandingBlock2D(256, conv_5, conv_4)
    conv_7 = ExpandingBlock2D(128, conv_6, conv_3)
    conv_8 = ExpandingBlock2D(64, conv_7, conv_2)
    conv_9 = ExpandingBlock2D(32, conv_8, conv_1)

    conv_10 = Conv2D(1, (1, 1), activation = 'sigmoid')(conv_9)

    model = Model(inputs = [input_tensor], outputs = [conv_10])
    return model

In [13]:
model = UNet()

In [14]:
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = dice_coeff_loss, metrics = [dice_coeff])

In [16]:
model_ckpt = ModelCheckpoint('final_unet_weights.h5', monitor='val_loss', save_best_only=True)
early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)
csv_logger = CSVLogger('training.log')

### Loading Dataset + Normalization

In [17]:
imgs_train = np.load('imgs_train.npy').astype('float32') 
masks_train = np.load('masks_train.npy').astype('float32') 

imgs_train = np.reshape(imgs_train, (imgs_train.shape[0], img_rows, img_cols, 1))
masks_train = np.reshape(masks_train, (masks_train.shape[0], img_rows, img_cols, 1))

In [18]:
mean = np.mean(imgs_train)
std = np.std(imgs_train)

In [19]:
imgs_train -= mean
imgs_train /= std

### Model Training

In [20]:
history=model.fit(imgs_train, masks_train, batch_size=256, epochs=20, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_ckpt, early_stop, csv_logger])

Train on 9184 samples, validate on 2297 samples
Epoch 1/20
9184/9184 [==============================] - 3820s 416ms/sample - loss: -0.1259 - dice_coeff: 0.1260 - val_loss: -0.0856 - val_dice_coeff: 0.0856
Epoch 2/20
9184/9184 [==============================] - 3816s 415ms/sample - loss: -0.1818 - dice_coeff: 0.1819 - val_loss: -0.0591 - val_dice_coeff: 0.0590
Epoch 3/20
9184/9184 [==============================] - 3813s 415ms/sample - loss: -0.2777 - dice_coeff: 0.2778 - val_loss: -0.0697 - val_dice_coeff: 0.0697
Epoch 4/20
9184/9184 [==============================] - 3805s 414ms/sample - loss: -0.4055 - dice_coeff: 0.4056 - val_loss: -0.0796 - val_dice_coeff: 0.0796
Epoch 5/20
9184/9184 [==============================] - 3809s 415ms/sample - loss: -0.5217 - dice_coeff: 0.5219 - val_loss: -0.1409 - val_dice_coeff: 0.1409